In [1]:
import pandas as pd

data = pd.read_csv(r'C:\Users\francischen\OneDrive - Carleton University\mas thesis\datasets\RawWDI.csv', encoding='iso-8859-1')
value_mapping = {
    "GDP per capita (constant 2015 US$)": "GDP pc",
    "GDP per capita growth (annual %)": "ln_GDPpc",
    "Taxes on income, profits and capital gains (% of revenue)": "tax",
    "Unemployment, total (% of total labor force) (national estimate)": "unemp",
    "Age dependency ratio, old (% of working-age population)": "age_dr",
    "Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)": "labor",
    "GDP deflator (base year varies by country)": "gdp_deflator"
    }

data["Series Name"] = data["Series Name"].map(value_mapping)
data = data.drop(['Country Code', 'Series Code'], axis=1)
#data = data.drop(data.index[1736: 1753], inplace=True)

data = pd.DataFrame(data)
num_rows_to_drop = 18
data = data[:-num_rows_to_drop] 
index_to_drop = list(range(6,1735,8))
data = data.drop(index_to_drop)

data.head(20)

,Country Name,Series Name,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Afghanistan,GDP pc,..,..,..,..,..,..,..,..,...,608.3863319,602.5166003,592.4761648,590.4273674,588.9300803,578.9855448,584.3865153,553.036131,426.2294013,..
1,Afghanistan,ln_GDPpc,..,..,..,..,..,..,..,..,...,2.002522145,-0.964803329,-1.666416415,-0.345802497,-0.253593782,-1.688576593,0.932833389,-5.364665931,-22.92919443,..
2,Afghanistan,tax,..,..,..,..,..,..,..,..,...,3.617021956,4.876052454,5.998665108,3.969484823,4.530615623,..,..,..,..,..
3,Afghanistan,unemp,..,..,..,..,..,..,..,..,...,..,7.91,..,..,11.18,..,..,11.71,5.58,..
4,Afghanistan,age_dr,5.100901063,5.078087919,5.049558481,5.022020416,4.99971913,4.985287472,4.9791624,4.976930721,...,4.737487477,4.690796787,4.644245638,4.606350887,4.57268661,4.551607034,4.53690098,4.494552727,4.437871825,4.395300155
5,Afghanistan,labor,..,..,..,..,..,..,..,..,...,48.176,48.268,48.358,48.449,48.541,48.636,48.743,42.609,..,..
7,Afghanistan,gdp_deflator,..,..,..,..,..,..,..,..,...,99.24148949,99.8041337,102.2469027,100,102.4036562,104.5247929,111.3413562,120.0501064,120.6232604,..
8,Albania,GDP pc,..,..,..,..,..,..,..,..,...,3780.699203,3855.760754,3952.803584,4090.372739,4249.820061,4431.555607,4543.387723,4418.660885,4857.111955,5158.509128
9,Albania,ln_GDPpc,..,..,..,..,..,..,..,..,...,1.187234353,1.985388092,2.516827054,3.480293217,3.898112267,4.276311551,2.523540843,-2.745238678,9.922713708,6.205275394
10,Albania,tax,..,..,..,..,..,..,..,..,...,14.33115005,13.81368207,14.91900192,11.48171936,15.53039227,16.68116194,18.37448413,15.36713044,13.5582784,..


In [78]:
# create a new dataframe df to store new format of data
inde = data.iloc[0:7, 1:2]  
old_cols = list(data.columns)[:2]  # Get the existing column names for country name and year
add_cols = inde.values.flatten()  # Flatten the 'inde' values
new_cols = old_cols + list(add_cols)  # Combine old and new column names
df = pd.DataFrame(columns=new_cols)

for country in data['Country Name'].unique():
    # Expand each country's data for the years 1961 to 2022
    country_data = pd.DataFrame({header: [None]*62 for header in new_cols})  # Creating a DataFrame with None values
    
    # Assign specific values to 'Country Name' and 'year' columns
    country_data['Country Name'] = country
    country_data['Year'] = list(range(1961, 2023))

    # Concatenate the new country data with the existing DataFrame
    df = pd.concat([df, country_data], ignore_index=True)
df = df.drop(["Series Name"], axis = 1)

column = df.columns.tolist()
column.remove('Year')
column = column[:1] + ['Year'] + column[1:]
df = df[column]
df.Year = df['Year'].astype('int')
  
for i in range(len(data['Country Name'].unique())):
    macro = data.iloc[i*7:(i+1)*7, 3:].transpose()
    df.iloc[i*62:(i+1)*62,2:]=macro
    
for index in range(2, 9):
    column_name = df.columns[index]
    new_column_name = f"{column_name}_lag"
    df[new_column_name] = "Nan"

data = data.reset_index(drop=True)
for i in range(len(data['Country Name'].unique())):
    df.iloc[i*62, 9:16] = data.iloc[i*7:(i+1)*7, 2]
    
for j in range(1, len(df)-60, 62):
    for k in range(j, j + 61):
        df.iloc[k, 9:16] = df.iloc[k-1, 2:9]

# add year effects into df
years = list(range(1961, 2023))
year_data = pd.DataFrame()
for year in years:
    year_data[f'year{year}'] = df['Year'].apply(lambda x: 1 if x == year else 0)
df = pd.concat([df, year_data], axis=1)

df.iloc[55:65, [0,1,9,15]]

,Country Name,Year,GDP pc,ln_GDPpc,tax,unemp,age_dr,labor,gdp_deflator,GDP pc_lag,...,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021,year2022
55,Afghanistan,2016,590.4273674,-0.345802497,3.969484823,..,4.606350887,48.449,100,592.4761648,...,0,0,0,1,0,0,0,0,0,0
56,Afghanistan,2017,588.9300803,-0.253593782,4.530615623,11.18,4.57268661,48.541,102.4036562,590.4273674,...,0,0,0,0,1,0,0,0,0,0
57,Afghanistan,2018,578.9855448,-1.688576593,..,..,4.551607034,48.636,104.5247929,588.9300803,...,0,0,0,0,0,1,0,0,0,0
58,Afghanistan,2019,584.3865153,0.932833389,..,..,4.53690098,48.743,111.3413562,578.9855448,...,0,0,0,0,0,0,1,0,0,0
59,Afghanistan,2020,553.036131,-5.364665931,..,11.71,4.494552727,42.609,120.0501064,584.3865153,...,0,0,0,0,0,0,0,1,0,0
60,Afghanistan,2021,426.2294013,-22.92919443,..,5.58,4.437871825,..,120.6232604,553.036131,...,0,0,0,0,0,0,0,0,1,0
61,Afghanistan,2022,..,..,..,..,4.395300155,..,..,426.2294013,...,0,0,0,0,0,0,0,0,0,1
62,Albania,1961,..,..,..,..,10.22378361,..,..,..,...,0,0,0,0,0,0,0,0,0,0
63,Albania,1962,..,..,..,..,10.21084124,..,..,..,...,0,0,0,0,0,0,0,0,0,0
64,Albania,1963,..,..,..,..,10.19120078,..,..,..,...,0,0,0,0,0,0,0,0,0,0


In [89]:
df.to_csv("RawWDI1", sep=',', index=True, encoding='iso-8859-1')

In [84]:
import os
path = os.getcwd()+"/new"
df.to_csv(path)

In [62]:
#for index in range(2, 9):
#df.iloc[len(df)-70:len(df)-55, :]
t=30
r2=0.1
r1=0.05
pe =list(range(t))
250*(sum((1+r2)**i-(1+r1)**i for i in pe))

24513.793796462764